## Google Colab Prepare

In [1]:
import os
os.makedirs('source', exist_ok=True)
os.makedirs('target', exist_ok=True)

In [2]:
from os.path import join
from scripts.data_management import EuroParlManager, Opus100Manager, FloresPlusManager
entries = {
    'opus': {'dm': Opus100Manager(), 'pairs': Opus100Manager.get_pairs()},
    'ep': {'dm': EuroParlManager(), 'pairs': EuroParlManager.get_pairs()},
    'flores': {'dm':FloresPlusManager(), 'pairs':FloresPlusManager.get_pairs()}
}   

In [3]:
for ent, content in entries.items():
    dm = content['dm']
    pairs = content['pairs']
    for pair in pairs:
        s,t = pair
        filename = f'{ent}-{s}-{t}.txt'
        src_path = join('source', filename)
        tgt_path = join('target', filename)
        src_sents, tgt_sents = dm.get_sentence_pairs(s, t, num_of_sents=400)
        with open(src_path, 'w') as f_src, open(tgt_path, 'w') as f_tgt:
            for src, tgt in zip(src_sents, tgt_sents):
                print(src, file=f_src)
                print(tgt, file=f_tgt)

### Example Usage

In [4]:
import sys
from contextlib import contextmanager
from io import StringIO

@contextmanager
def captureStdOut(output):
    stdout = sys.stdout
    sys.stdout = output
    try:
        yield
    finally:
        sys.stdout = stdout

In [5]:
from scripts.post_process import load_sents_from_file, align_sents
import time
import logging
fmt = '%(levelname)s: %(asctime)s - %(message)s'
datefmt = '%Y-%m-%d %H:%M:%S'

logging.basicConfig(filename='output.log', level=logging.DEBUG, format=fmt, datefmt=datefmt, force=True)

examples = ['ep-gpt-de-en', 'flores-gpt-de-en']
for ex in examples:
    mt_sents = load_sents_from_file(ex, folder='translations')
    dataset, translator, s, t = ex.split('-')
    src_sents = load_sents_from_file(f'{dataset}-{s}-{t}', folder='source')

    start = time.time()
    out = StringIO()
    with captureStdOut(out):
        align_sents(
            filename=ex,
            src_lang=s,
            tgt_lang=t,
            src_sents=src_sents[:10],
            tgt_sents=mt_sents[:10],
            folder_path='tmp',
        )
    end = time.time()
    logging.info(f'Alignment: {ex}')
    logging.info(f'Duration: {end-start}')
    logging.info(f'Stdout: \n{out.getvalue()}')

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from scripts.post_process import load_aligned_sents_from_file
s, m = load_aligned_sents_from_file('ep-gpt-de-en', folder='tmp')
s, m

(['Wiederaufnahme der Sitzungsperiode',
  'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.',
  'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten.',
  'Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.',
  'Im Parlament besteht der Wunsch nach einer Aussprache im Verlauf dieser Sitzungsperiode in den nächsten Tagen.',
  'Heute möchte ich Sie bitten - das ist auch der Wunsch einiger Kolleginnen und Kollegen -, allen Opfern der Stürme, insbesondere in den verschiedenen Ländern der Europäischen Union, in einer Schweigeminute zu gedenken.',
  'Ich bitte Sie, sich zu einer Schweigeminute zu erheben.',
  '(Das Parlament erhebt sich zu einer Schweigeminute.)',
  'Frau Präsidentin, zur Geschäftsordnung.',
  'Wie Sie sicher aus der Presse und 

* Thus, we only need to zip `source`, `translations` and `target` folder and uploud it with the `post_process.py` script to Google Colab.